# Large Circuit Partitioning Benchmark
This notebook builds composite circuits that combine structured subroutines to exercise QuASAr's partitioning capabilities:

* **Mixed backend subsystems** via `mixed_backend_subsystems`, combining a Clifford GHZ prefix, an MPS-friendly QAOA mid-block, and a dense random suffix that forces conversions between tableau, MPS, and statevector simulations.
* **Stim-to-DD conversion** via `stim_to_dd_circuit`, preparing multiple stabiliser subsystems before a global non-Clifford layer triggers decision-diagram conversion.

We inspect the resulting partitions, conversion layers, and execution metrics produced by QuASAr.

In [ ]:
import os
import sys
from pathlib import Path

os.environ['QUASAR_QUICK_MAX_QUBITS'] = '16'
os.environ['QUASAR_QUICK_MAX_GATES'] = '200'
os.environ['QUASAR_QUICK_MAX_DEPTH'] = '20'

ROOT = Path().resolve().parent.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


In [ ]:
from benchmarks.partition_circuits import mixed_backend_subsystems, stim_to_dd_circuit
from quasar.analyzer import CircuitAnalyzer
from quasar.simulation_engine import SimulationEngine

mixed = mixed_backend_subsystems(ghz_width=5, qaoa_width=5, qaoa_layers=2, random_width=4, seed=13)
stim_conversion = stim_to_dd_circuit(num_groups=3, group_size=3, entangling_layer=True)

CASES = [
    ("Mixed Backend Subsystems", mixed),
    ("Stim-to-DD Conversion", stim_conversion),
]


In [ ]:
def describe_partitions(name, partitions):
    if not partitions:
        print(f"[{name}] No partitions identified.")
        return
    for idx, part in enumerate(partitions, start=1):
        backend_name = getattr(part.backend, 'name', str(part.backend))
        history_len = len(part.history)
        subsystem_desc = _format_subsystems(part.subsystems)
        print(
            f"[{name}] Partition {idx}: backend={backend_name}, copies={part.multiplicity}, "
            f"subsystems=[{subsystem_desc}], gates={history_len}"
        )

In [ ]:
from itertools import count

def simulate_prepared(circuit, plan, analysis, name):
    total_steps = len(getattr(plan, 'explicit_steps', plan.steps))
    step_counter = count(1)

    def monitor(step, observed, estimated):
        idx = next(step_counter)
        backend_name = getattr(step.backend, 'name', str(step.backend))
        gate_count = step.end - step.start
        est_time = getattr(estimated, 'time', 0.0)
        obs_time = getattr(observed, 'time', 0.0)
        print(
            f"[{name}] Step {idx}/{total_steps} on {backend_name}: {gate_count} gate(s), est. {est_time:.2f}s, observed {obs_time:.2f}s."
        )

    print(f"[{name}] Executing prepared plan with {total_steps} step(s)...")
    conversions = list(getattr(plan, 'explicit_conversions', ()))
    if conversions:
        print(f"[{name}] Planned conversions:")
        describe_conversions(name, conversions)
    ssd, metrics = engine.scheduler.run(
        circuit,
        plan,
        analysis=analysis,
        instrument=True,
        monitor=monitor,
    )
    runtime = metrics.cost.time
    peak_memory = (metrics.cost.memory / (1024 ** 3)) if metrics.cost.memory else 0.0
    print(
        f"[{name}] Simulation complete in {runtime:.2f}s with peak memory {peak_memory:.2f} GiB and {metrics.backend_switches} backend switch(es)."
    )
    print(f"[{name}] Final SSD partitions: {len(ssd.partitions)}")
    return ssd, metrics

simulation_outputs = {}

for label, circuit in CASES:
    simulation_outputs[label] = simulate_prepared(circuit, plans[label], analyses[label], label)
